In [ ]:
## in python 3.9.13 version install the following:

# pip install requests grpcio flask gunicorn keras-image-helper tensorflow tensorflow-serving-api==2.7.0 tensorflow-protobuf protobuf keras-image-helper

In [1]:
import grpc
import tensorflow as tf

2024-12-25 18:29:16.884278: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-25 18:29:16.888312: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-25 18:29:16.900250: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735122556.921134   53628 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735122556.927253   53628 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 18:29:16.949115: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [3]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [4]:
from keras_image_helper import create_preprocessor

In [5]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [6]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [7]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [8]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [10]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [12]:
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.87986457
    float_val: -4.75631189
    float_val: -2.35953331
    float_val: -1.08926427
    float_val: 9.90378475
    float_val: -2.82618117
    float_val: -3.64831042
    float_val: 3.24115634
    float_val: -2.61209583
    float_val: -4.852036
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [14]:
preds = pb_response.outputs['dense_7'].float_val

In [15]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds))

{'dress': -1.8798645734786987,
 'hat': -4.756311893463135,
 'longsleeve': -2.3595333099365234,
 'outwear': -1.0892642736434937,
 'pants': 9.90378475189209,
 'shirt': -2.826181173324585,
 'shoes': -3.648310422897339,
 'shorts': 3.2411563396453857,
 'skirt': -2.612095832824707,
 't-shirt': -4.852035999298096}